# Reversibility Index - 2 Inputs

In [173]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="View/Hide Code"></form>''')

In [174]:
%matplotlib widget
# %matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.gridspec import GridSpec
from palettable.colorbrewer.sequential import YlGnBu_9
from pandas import DataFrame
from IPython.display import display, Markdown, Latex


class F(object):
    def __init__(self,b_g,a_g,n,w,f):                
        self.beta_grid = b_g # Beta grid
        self.alpha_grid = a_g # Alpha grid
        self.distance = (self.alpha_grid - self.beta_grid)/np.sqrt(2) # Grid of the distance from diagonal
        self.N = n # Number of weights / Maximum input of the other expression 
        self.u_max = np.max(self.beta_grid) # Maximum input / Number of weights of the other expression 
        self.weight = w # 3D array of weights
        
        # Plotting stuff
        self.axe_plot = {}
        self.figure_num = f
    
    # R index functions
    
    def I_mu_D(self,mu): 
        return np.nansum(mu*self.distance)
    def I_mu(self,mu):
        return np.nansum(mu)
    def R_exp(self):
        self.numerator = 0
        self.denumerator = 0
        for i in range(self.N):
            self.numerator = self.numerator + self.I_mu_D(self.weight[i,:,:])
            self.denumerator = self.denumerator + self.I_mu(self.weight[i,:,:])
        return (self.numerator*np.sqrt(2))/((self.denumerator*self.u_max))
    
    
    
    def plot_weight(self):
        #Plot
        self.fig = plt.figure(self.figure_num, figsize=(1,1))
        w_r = [1 for i in range(self.N)]
        gs = GridSpec(1, self.N,
                             left=0.0175, right=1,  
                             bottom=0.25, top=1,
                             wspace=0.3, hspace=0)
        for i in range(self.N):
            self.axe_plot.update({i:plt.subplot(gs[0, i])})
        for i in range(self.N):
            self.axe_plot[i].imshow(
                                    self.weight[i,:,:],
                                    cmap=YlGnBu_9.mpl_colormap,
                                    vmax=np.nanmax(self.weight[i,:,:]),
                                    vmin=np.nanmin(self.weight[i,:,:])
                                   )
            for b in range(len(self.beta_grid[0,:])):  
                for a in range(len(self.alpha_grid[0,:])): 
                    self.axe_plot[i].annotate(np.around(self.weight[i,a,b],2),
                                              (self.beta_grid[a,b]-0.25, self.alpha_grid[a,b]),size=6,color="red")
            self.axe_plot[i].set_aspect('equal', adjustable='box')
            self.axe_plot[i].spines['top'].set_visible(False)
            self.axe_plot[i].spines['right'].set_visible(False)
            self.axe_plot[i].set_title(i)
            self.axe_plot[i].set_xlim([-0.5,self.u_max+0.5])
            self.axe_plot[i].set_ylim([-0.5,self.u_max+0.5])
            # Calculating the R of each weight as individual
            self.axe_plot[i].set_xlabel(
                                            r'$\sum(\mu(\alpha,\beta)D(\alpha,\beta))$='
                                            +str(np.around(self.I_mu_D(self.weight[i,:,:]),2))
                                            +"\n"+r'$\sum(\mu(\alpha,\beta))$='
                                            +str(np.around(self.I_mu(self.weight[i,:,:]),2))
                                            +"\nR.I. = "
                                            +str(np.around(1-(self.I_mu_D((self.weight[i,:,:])*np.sqrt(2))/(I_mu(self.weight[i,:,:])*self.u_max)),4))
                                       
                                        ,size=9.5)
    def Macro_info(self):
            self.R_exp()
            if self.figure_num == 1:
                display(Markdown(r'$\sum_{k=0}^N \sum(\mu(\alpha,\beta,k)D(\alpha,\beta)) = $'
                                 +str(np.around(self.numerator,2))))
                display(Markdown(r'$\sum_{k=0}^N \sum(\mu(\alpha,\beta,k)) = $'
                                 +str(np.around(self.denumerator,2))))
                display(Markdown(r'$N = $'+str(float(self.N)-1)))
            else:
                display(Markdown(r'$\sum_{k=0}^M \sum(\mu(\alpha,\beta,k)D(\alpha,\beta)) = $'
                                 +str(np.around(self.numerator,2))))
                display(Markdown(r'$\sum_{k=0}^M \sum(\mu(\alpha,\beta,k)) = $'
                                 +str(np.around(self.denumerator,2))))
                display(Markdown(r'$M = $'+str(float(self.N)-1)))

                
def weights(method,a_v,b_v,maxU):
    alpha_grid = a_v
    beta_grid = b_v
    mu = np.zeros((len(alpha_grid[0]), len(alpha_grid[0])))
    mu =  {
           'uniform': np.where(alpha_grid>=beta_grid, maxU, np.nan),
           'linear': np.where(alpha_grid==beta_grid, maxU, np.nan),
           'top_heavy': np.where(alpha_grid>=beta_grid, alpha_grid, np.nan),
           'bottom_heavy': np.where(alpha_grid>=beta_grid, maxU-alpha_grid, np.nan),
           'right_heavy': np.where(alpha_grid>=beta_grid, beta_grid, np.nan),
           'left_heavy': np.where(alpha_grid>=beta_grid, maxU-beta_grid, np.nan),
           'center_light_alpha': np.where(alpha_grid>=beta_grid, np.abs(0.5*maxU-alpha_grid), np.nan),
           'center_light_beta': np.where(alpha_grid>=beta_grid, np.abs(0.5*maxU-beta_grid), np.nan),
           'single_line': np.where(np.logical_and(0.3*maxU<beta_grid, beta_grid<0.5*maxU), maxU, 0),
           'upper_left': np.where(np.logical_and(0.2*maxU>beta_grid, alpha_grid>0.6*maxU), maxU, 0)
    }[method]
    mu = np.where(alpha_grid>=beta_grid, mu, np.nan)
    return mu / np.nansum(mu)        



N1 = 3
N2 = 5

alpha_value1 = np.around(np.linspace(0, N2-1, N2),2) # Grid of beta and alpha
beta_value1 = np.around(np.linspace(0, N2-1, N2),2)
beta_grid1, alpha_grid1 = np.meshgrid(beta_value1, alpha_value1)
# Definding weights and stack them in a 3D array. The first axis represnet each whole weight
wA1 = weights("top_heavy",alpha_grid1,beta_grid1,N2)
wA2 = weights("center_light_alpha",alpha_grid1,beta_grid1,N2)
wA3 = weights("upper_left",alpha_grid1,beta_grid1,N2)
WA= np.stack((wA1,wA2,wA3))
f1 = F(beta_grid1,alpha_grid1,N1,WA,1)
f1.plot_weight()
f1.Macro_info()

alpha_value2 = np.around(np.linspace(0, N1-1, N1),2)
beta_value2 = np.around(np.linspace(0, N1-1, N1),2)
beta_grid2, alpha_grid2 = np.meshgrid(beta_value2, alpha_value2)
wB1 = weights("uniform",alpha_grid2,beta_grid2,N1)
wB2 = weights("top_heavy",alpha_grid2,beta_grid2,N1)
wB3 = weights("linear",alpha_grid2,beta_grid2,N1)
wB4 = weights("right_heavy",alpha_grid2,beta_grid2,N1)
wB5 = weights("left_heavy",alpha_grid2,beta_grid2,N1)
WB= np.stack((wB1,wB2,wB3,wB4,wB5))
f2 = F(beta_grid2,alpha_grid2,N2,WB,2)
f2.plot_weight()
f2.Macro_info()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

$\sum_{k=0}^N \sum(\mu(\alpha,\beta,k)D(\alpha,\beta)) = $4.88

$\sum_{k=0}^N \sum(\mu(\alpha,\beta,k)) = $3.0

$N = $2.0

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

$\sum_{k=0}^M \sum(\mu(\alpha,\beta,k)D(\alpha,\beta)) = $1.82

$\sum_{k=0}^M \sum(\mu(\alpha,\beta,k)) = $5.0

$M = $4.0

## Final Calculation - Substitution:

In [175]:
display(Markdown(r"$R_f = 2 - \frac{\sqrt{2}}{M}\frac{ \displaystyle\sum_k^N\sum_{\alpha\geq \beta}^M \mu(\alpha,\beta, k) \mathcal{D}(\alpha,\beta)}{\displaystyle\sum_k^N\sum_{\alpha\geq \beta}^M \mu(\alpha,\beta, k)}-\frac{\sqrt{2}}{N}\frac{ \displaystyle\sum_k^M\sum_{\alpha\geq \beta}^N \nu(\alpha,\beta, k) \mathcal{D}(\alpha,\beta)}{\displaystyle\sum_k^M\sum_{\alpha\geq \beta}^N \nu(\alpha,\beta, k)}$"))
display(Markdown(
r"$R_f = $"+str(2 - f1.R_exp()- f2.R_exp())
))
display(Markdown(
r"$\frac{R_f}{2}=$"+str((2 - f1.R_exp()- f2.R_exp())/2)
))


$R_f = 2 - \frac{\sqrt{2}}{M}\frac{ \displaystyle\sum_k^N\sum_{\alpha\geq \beta}^M \mu(\alpha,\beta, k) \mathcal{D}(\alpha,\beta)}{\displaystyle\sum_k^N\sum_{\alpha\geq \beta}^M \mu(\alpha,\beta, k)}-\frac{\sqrt{2}}{N}\frac{ \displaystyle\sum_k^M\sum_{\alpha\geq \beta}^N \nu(\alpha,\beta, k) \mathcal{D}(\alpha,\beta)}{\displaystyle\sum_k^M\sum_{\alpha\geq \beta}^N \nu(\alpha,\beta, k)}$

$R_f = $1.1674512987012988

$\frac{R_f}{2}=$0.5837256493506494